In [0]:
import numpy as np
import pandas as pd
import re
import tensorflow as tf
from tensorflow import keras

In [2]:
tf.__version__

'2.2.0-rc2'

In [3]:
from google.colab import drive

drive.mount('/content/drive/', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
reviews_df = pd.read_csv('/content/drive/My Drive/Amazon_Reviews.csv')

In [5]:
reviews_df['Score'].value_counts()

5    363122
4     80655
1     52268
3     42640
2     29769
Name: Score, dtype: int64

In [6]:
neg_df = reviews_df[reviews_df['Score'] <= 4]

neg_df = neg_df.dropna(subset=['Summary'])

neg_df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,4,1342051200,Nice Taffy,I got a wild hair for taffy and ordered this f...
12,13,B0009XLVG0,A327PCT23YH90,LT,1,1,1,1339545600,My Cats Are Not Fans of the New Food,My cats have been happily eating Felidae Plati...


## Load CSV of CFPB Complaints and Summaries

In [0]:
cfpb_df = pd.read_csv('/content/drive/My Drive/cfpb_summaries.csv')

sum_df = cfpb_df.dropna(subset=['Summary'])

# Build test_df
test_df = sum_df.sample(10, random_state=42)

test_df['Text'] = test_df['Consumer complaint narrative']

test_df = test_df[['Text', 'Summary']]

# Remove rows that were sampled for test_df
to_remove = test_df.index.tolist()

sum_df = sum_df.drop(to_remove)

test_df = test_df.reset_index(drop=True)

## Build Combined DataFrame w/ 'X' Copies of CFPB Text & Summaries

In [0]:
sum_df['Text'] = sum_df['Consumer complaint narrative']

tmp_df = sum_df[['Text', 'Summary']]

tmp_df_2 = neg_df[['Text', 'Summary']]

comb_df = tmp_df.append(tmp_df_2)

comb_df.reset_index(inplace=True, drop=True)

for i in range(20):

  comb_df = tmp_df.append(comb_df).reset_index(drop=True)

In [10]:
comb_df.shape

(208896, 2)

## Clean Texts

In [11]:
clean = [re.sub('[^A-Za-z.,\s\']', '', text) for text in comb_df['Text']]

split_word_nars = [nar.split() for nar in clean]

"""Contractions Import"""

import sys
sys.path.append('/content/drive/My Drive')
from english_contractions import replace_contraction

""" Stop Words Import """

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stops = set(stopwords.words('english'))

""" Loops """

new_words = []

for nar in split_word_nars:

  nar_words = []

  for word in nar:

    if re.search('\w+[.]', word):

      splitted = word.split('.')

      tmp_words = replace_contraction(splitted[0].lower())

      for w in tmp_words.split():

        nar_words.append(w)

      nar_words.append('.')
    
    elif re.search('\w+[,]', word):
      
      splitted = word.split(',')

      tmp_words = replace_contraction(splitted[0].lower())

      for w in tmp_words.split():

        nar_words.append(w)

      nar_words.append(',')
    
    elif re.match('[.]', word):
      
      placeholder = 1
    
    else:

      tmp_words = replace_contraction(word)

      for w in tmp_words.split():

        nar_words.append(w)
  
  nar_words = [word for word in nar_words if not word in stops]

  new_words.append(' '.join(nar_words))

texts = new_words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Clean Summaries

In [0]:
clean = [re.sub('[^A-Za-z.,\s\']', '', text) for text in comb_df['Summary']]

split_word_nars = [nar.split() for nar in clean]

"""Contractions Import"""

import sys
sys.path.append('/content/drive/My Drive')
from english_contractions import replace_contraction

""" Loops """

new_words = []

for nar in split_word_nars:

  nar_words = []

  for word in nar:

    if re.search('\w+[.]', word):

      splitted = word.split('.')

      tmp_words = replace_contraction(splitted[0].lower())

      for w in tmp_words.split():

        nar_words.append(w)

      nar_words.append('.')
    
    elif re.search('\w+[,]', word):
      
      splitted = word.split(',')

      tmp_words = replace_contraction(splitted[0].lower())

      for w in tmp_words.split():

        nar_words.append(w)

      nar_words.append(',')
    
    elif re.match('[.]', word):
      
      placeholder = 1
    
    else:

      tmp_words = replace_contraction(word)

      for w in tmp_words.split():

        nar_words.append(w)

  new_words.append(' '.join(nar_words))

summaries = new_words

## Clean Test Text and Summaries

In [13]:
# Texts

clean = [re.sub('[^A-Za-z.,\s\']', '', text) for text in test_df['Text']]

split_word_nars = [nar.split() for nar in clean]

"""Contractions Import"""

import sys
sys.path.append('/content/drive/My Drive')
from english_contractions import replace_contraction

""" Stop Words Import """

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stops = set(stopwords.words('english'))

""" Loops """

new_words = []

for nar in split_word_nars:

  nar_words = []

  for word in nar:

    if re.search('\w+[.]', word):

      splitted = word.split('.')

      tmp_words = replace_contraction(splitted[0].lower())

      for w in tmp_words.split():

        nar_words.append(w)

      nar_words.append('.')
    
    elif re.search('\w+[,]', word):
      
      splitted = word.split(',')

      tmp_words = replace_contraction(splitted[0].lower())

      for w in tmp_words.split():

        nar_words.append(w)

      nar_words.append(',')
    
    elif re.match('[.]', word):
      
      placeholder = 1
    
    else:

      tmp_words = replace_contraction(word)

      for w in tmp_words.split():

        nar_words.append(w)
  
  nar_words = [word for word in nar_words if not word in stops]

  new_words.append(' '.join(nar_words))

test_texts = new_words

# Summaries

clean = [re.sub('[^A-Za-z.,\s\']', '', text) for text in test_df['Summary']]

split_word_nars = [nar.split() for nar in clean]

new_words = []

for nar in split_word_nars:

  nar_words = []

  for word in nar:

    if re.search('\w+[.]', word):

      splitted = word.split('.')

      tmp_words = replace_contraction(splitted[0].lower())

      for w in tmp_words.split():

        nar_words.append(w)

      nar_words.append('.')
    
    elif re.search('\w+[,]', word):
      
      splitted = word.split(',')

      tmp_words = replace_contraction(splitted[0].lower())

      for w in tmp_words.split():

        nar_words.append(w)

      nar_words.append(',')
    
    elif re.match('[.]', word):
      
      placeholder = 1
    
    else:

      tmp_words = replace_contraction(word)

      for w in tmp_words.split():

        nar_words.append(w)

  new_words.append(' '.join(nar_words))

test_summaries = new_words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Build Clean Df

In [14]:
comb_df['Cleaned_text'], comb_df['Cleaned_summary'] = texts, summaries

clean_df = comb_df[['Cleaned_text', 'Cleaned_summary']]

target_texts = []

for target_text in clean_df['Cleaned_summary']:

  target_text = '_START_ ' + str(target_text) + ' _END_'

  target_texts.append(target_text)

clean_df['Summary'] = target_texts

# test_df:
test_df['Cleaned_text'], test_df['Cleaned_summary'] = test_texts, test_summaries

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


## Train/Val Split

In [0]:
max_len_text = 100

max_len_summary = 10

from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(clean_df['Cleaned_text'], clean_df['Summary'], test_size=0.15, random_state=0, shuffle=True)

In [16]:
clean_df['Summary'].sample(1)

80871    _START_ Total rip off _END_
Name: Summary, dtype: object

## Tokenize Text

### Reviews

In [17]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

x_tokenizer = tf.keras.preprocessing.text.Tokenizer()

x_tokenizer.fit_on_texts(list(X_train))

# convert text sequences into integer sequences
X_train = x_tokenizer.texts_to_sequences(X_train)
X_val = x_tokenizer.texts_to_sequences(X_val)

# padding zero up to maximum length
X_train = pad_sequences(X_train, maxlen=max_len_text, padding='post')
X_val = pad_sequences(X_val, maxlen=max_len_text, padding='post')

x_voc_size = len(x_tokenizer.word_index) + 1

Using TensorFlow backend.


### Summaries

In [0]:
y_tokenizer = tf.keras.preprocessing.text.Tokenizer()

y_tokenizer.fit_on_texts(list(y_train))

# convert summary sequences into integer sequences
y_train = y_tokenizer.texts_to_sequences(y_train)
y_val = y_tokenizer.texts_to_sequences(y_val)

# padding zero upto maximum length
y_train = pad_sequences(y_train, maxlen=max_len_summary, padding='post')
y_val = pad_sequences(y_val, maxlen=max_len_summary, padding='post')

y_voc_size = len(y_tokenizer.word_index) + 1

## AttentionLayer Class

In [0]:
import os
from tensorflow.python.keras.layers import Layer
from tensorflow.python.keras import backend as K


class AttentionLayer(Layer):
    """
    This class implements Bahdanau attention (https://arxiv.org/pdf/1409.0473.pdf).
    There are three sets of weights introduced W_a, U_a, and V_a
     """

    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        # Create a trainable weight variable for this layer.

        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='uniform',
                                   trainable=True)

        super(AttentionLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs, verbose=False):
        """
        inputs: [encoder_output_sequence, decoder_output_sequence]
        """
        assert type(inputs) == list
        encoder_out_seq, decoder_out_seq = inputs
        if verbose:
            print('encoder_out_seq>', encoder_out_seq.shape)
            print('decoder_out_seq>', decoder_out_seq.shape)

        def energy_step(inputs, states):
            """ Step function for computing energy for a single decoder state """

            assert_msg = "States must be a list. However states {} is of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            """ Some parameters required for shaping tensors"""
            en_seq_len, en_hidden = encoder_out_seq.shape[1], encoder_out_seq.shape[2]
            de_hidden = inputs.shape[-1]

            """ Computing S.Wa where S=[s0, s1, ..., si]"""
            # <= batch_size*en_seq_len, latent_dim
            reshaped_enc_outputs = K.reshape(encoder_out_seq, (-1, en_hidden))
            # <= batch_size*en_seq_len, latent_dim
            W_a_dot_s = K.reshape(K.dot(reshaped_enc_outputs, self.W_a), (-1, en_seq_len, en_hidden))
            if verbose:
                print('wa.s>',W_a_dot_s.shape)

            """ Computing hj.Ua """
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)  # <= batch_size, 1, latent_dim
            if verbose:
                print('Ua.h>',U_a_dot_h.shape)

            """ tanh(S.Wa + hj.Ua) """
            # <= batch_size*en_seq_len, latent_dim
            reshaped_Ws_plus_Uh = K.tanh(K.reshape(W_a_dot_s + U_a_dot_h, (-1, en_hidden)))
            if verbose:
                print('Ws+Uh>', reshaped_Ws_plus_Uh.shape)

            """ softmax(va.tanh(S.Wa + hj.Ua)) """
            # <= batch_size, en_seq_len
            e_i = K.reshape(K.dot(reshaped_Ws_plus_Uh, self.V_a), (-1, en_seq_len))
            # <= batch_size, en_seq_len
            e_i = K.softmax(e_i)

            if verbose:
                print('ei>', e_i.shape)

            return e_i, [e_i]

        def context_step(inputs, states):
            """ Step function for computing ci using ei """
            # <= batch_size, hidden_size
            c_i = K.sum(encoder_out_seq * K.expand_dims(inputs, -1), axis=1)
            if verbose:
                print('ci>', c_i.shape)
            return c_i, [c_i]

        def create_inital_state(inputs, hidden_size):
            # We are not using initial states, but need to pass something to K.rnn funciton
            fake_state = K.zeros_like(inputs)  # <= (batch_size, enc_seq_len, latent_dim
            fake_state = K.sum(fake_state, axis=[1, 2])  # <= (batch_size)
            fake_state = K.expand_dims(fake_state)  # <= (batch_size, 1)
            fake_state = K.tile(fake_state, [1, hidden_size])  # <= (batch_size, latent_dim
            return fake_state

        fake_state_c = create_inital_state(encoder_out_seq, encoder_out_seq.shape[-1])
        fake_state_e = create_inital_state(encoder_out_seq, encoder_out_seq.shape[1])  # <= (batch_size, enc_seq_len, latent_dim

        """ Computing energy outputs """
        # e_outputs => (batch_size, de_seq_len, en_seq_len)
        last_out, e_outputs, _ = K.rnn(
            energy_step, decoder_out_seq, [fake_state_e],
        )

        """ Computing context vectors """
        last_out, c_outputs, _ = K.rnn(
            context_step, e_outputs, [fake_state_c],
        )

        return c_outputs, e_outputs

    def compute_output_shape(self, input_shape):
        """ Outputs produced by the layer """
        return [
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[1][2])),
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[0][1]))
        ]

## Build Model

In [39]:
import tensorflow_addons as tfa

n_neurons = 256

embedding_dim = 512

dropout_rate = 0.25

# Encoder 
encoder_inputs = tf.keras.layers.Input(shape=[max_len_text,])
enc_emb_layer = tf.keras.layers.Embedding(x_voc_size, embedding_dim, trainable=True)
enc_emb = enc_emb_layer(encoder_inputs)

# LSTM 1 
encoder_lstm_1 = tf.keras.layers.LSTM(n_neurons, return_sequences=True, return_state=True,
                                      dropout=dropout_rate, recurrent_dropout=dropout_rate)
encoder_output_1, state_h1, state_c1 = encoder_lstm_1(enc_emb)

# LSTM 2 
encoder_lstm_2 = tf.keras.layers.LSTM(n_neurons, return_sequences=True, return_state=True,
                                      dropout=dropout_rate, recurrent_dropout=dropout_rate)
encoder_output_2, state_h2, state_c2 = encoder_lstm_2(encoder_output_1)

# LSTM 3 
encoder_lstm_3 = tf.keras.layers.LSTM(n_neurons, return_sequences=True, return_state=True,
                                      dropout=dropout_rate, recurrent_dropout=dropout_rate)
encoder_outputs, state_h, state_c = encoder_lstm_3(encoder_output_2)

# Set up the decoder.
decoder_inputs = tf.keras.layers.Input(shape=[None,])
dec_emb_layer = tf.keras.layers.Embedding(y_voc_size, embedding_dim, trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)

# LSTM using encoder_states as initial state
decoder_lstm = tf.keras.layers.LSTM(n_neurons, return_sequences=True, return_state=True,
                                    dropout=0.25, recurrent_dropout=0.25)
decoder_output_1, decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb, initial_state=[state_h, state_c])

# Attention Layer
attn_layer = AttentionLayer(name='attention_layer')
attn_out, attn_states = attn_layer([encoder_outputs, decoder_output_1])

# Concat attention output and decoder LSTM output
decoder_concat_input = tf.keras.layers.Concatenate(axis=-1, name='concat_layer')([decoder_output_1, attn_out])

# Dense layer 1
decoder_concat_input = tf.keras.layers.Dropout(0.5)(decoder_concat_input)
decoder_dense_1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1024, activation='sigmoid'))
decoder_output_2 = decoder_dense_1(decoder_concat_input)

# Dense output
decoder_dense = tf.keras.layers.Dense(y_voc_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_output_2)

# Define the model
model = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 100, 512)     50712576    input_6[0][0]                    
__________________________________________________________________________________________________
lstm_4 (LSTM)                   [(None, 100, 256), ( 787456      embedding_2[0][0]                
__________________________________________________________________________________________________
input_7 (InputLayer)            [(None, None)]       0                                            
____________________________________________________________________________________________

## Train Model

### Build Target Set

In [0]:
y_train_post = y_train[:,:-1]

y_train_post_2 = y_train.reshape(y_train.shape[0], y_train.shape[1], -1)[:,1:]

y_val_post = y_val[:,:-1]

y_val_post_2 = y_val.reshape(y_val.shape[0], y_val.shape[1], -1)[:,1:]

### Establish Callbacks

In [41]:
# Early Stopping

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)

# Checkpointing Model Weights

import os

checkpoint_path = 'checkpoints/cp-{epoch:03d}.ckpt'

checkpoint_dir = os.path.dirname(checkpoint_path)

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, verbose=1,
                                                   save_weights_only=True, period=1)

latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)

latest_checkpoint

'checkpoints/cp-010.ckpt'

### Train Model

In [0]:
batch_size = 256

epochs = 10

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

model.load_weights(latest_checkpoint)

history = model.fit([X_train, y_train_post], y_train_post_2,
                    validation_data=([X_val, y_val_post], y_val_post_2),
                    batch_size=batch_size, epochs=epochs, callbacks=[early_stop, checkpoint_cb])

Epoch 1/10
 20/694 [..............................] - ETA: 14:27 - loss: 1.6381

## Save Model and Tokenizers

### Model

In [0]:
drive_path = '/content/drive/'

file_path = 'My Drive/saved_keras_rnns/'

name = 'enc_dec_main_u3.h5'

model.save(drive_path + file_path + name)

### Tokenizers

In [0]:
import pickle

name = 'tok_enc_dec_main_u3.pkl'

pickle.dump((x_tokenizer, y_tokenizer), open(drive_path + file_path + name, 'wb'))

## Load Model and Tokenizers

### Model

In [60]:
drive_path = '/content/drive/'

file_path = 'My Drive/saved_keras_rnns/'

name = 'enc_dec_main.h5'

model = keras.models.load_model(drive_path + file_path + name, custom_objects={'AttentionLayer': AttentionLayer})

### Tokenizers

In [0]:
name = 'tok_enc_dec_main.pkl'

tokenizers_tuple = pickle.load(open(drive_path + file_path + name, 'rb'))

## Get Word Indices

In [0]:
reverse_source_word_index = x_tokenizer.index_word

reverse_target_word_index = y_tokenizer.index_word

target_word_index = y_tokenizer.word_index

## Model

In [0]:
# encoder inference
encoder_model = tf.keras.Model(inputs=encoder_inputs, outputs=[encoder_outputs, state_h, state_c])

# decoder inference
# Below tensors will hold the states of the previous time step
decoder_state_input_h = tf.keras.layers.Input(shape=(n_neurons,))
decoder_state_input_c = tf.keras.layers.Input(shape=(n_neurons,))
decoder_hidden_state_input = tf.keras.layers.Input(shape=(max_len_text, n_neurons))

# Get the embeddings of the decoder sequence
dec_emb_2 = dec_emb_layer(decoder_inputs)

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs_2, state_h2, state_c2 = decoder_lstm(dec_emb_2, initial_state=[decoder_state_input_h, decoder_state_input_c])

# Attention inference
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs_2])
decoder_inf_concat = tf.keras.layers.Concatenate(axis=-1, name='concat')([decoder_outputs_2, attn_out_inf])

# Dense layer
decoder_outputs_3 = decoder_dense_1(decoder_inf_concat)

# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs_4 = decoder_dense(decoder_outputs_3)

# Final decoder model
decoder_model = tf.keras.Model([decoder_inputs] + [decoder_hidden_state_input, decoder_state_input_h, decoder_state_input_c],
                               [decoder_outputs_4] + [state_h2, state_c2])

## Decode Sequence

In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))

    # Chose the 'start' word as the first word of the target sequence
    target_seq[0, 0] = target_word_index['start']

    stop_condition = False

    decoded_sentence = ''

    while not stop_condition:

        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])

        sampled_token = reverse_target_word_index[sampled_token_index]

        if sampled_token != 'end':

            decoded_sentence += ' ' + sampled_token

            # Exit condition: either hit max length or find stop word.
        if (sampled_token == 'end') or (len(decoded_sentence.split()) >= (max_len_summary-1)):
                
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))

        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence

In [0]:
def seq2summary(input_seq):

    new_string = ''

    for i in input_seq:

      if ((i != 0 and i != target_word_index['start']) and i != target_word_index['end']):

        new_string = new_string + reverse_target_word_index[i] + ' '

    return new_string

def seq2text(input_seq):

    new_string = ''

    for i in input_seq:

      if i != 0:

        new_string = new_string + reverse_source_word_index[i] + ' '
        
    return new_string

## Show Predictions

### Tokenize Test Text

In [0]:
test_text = x_tokenizer.texts_to_sequences(test_df['Cleaned_text'])

test_text = pad_sequences(test_text, maxlen=max_len_text, padding='post')

test_sum = y_tokenizer.texts_to_sequences(test_df['Cleaned_summary'])

test_sum = pad_sequences(test_sum, maxlen=max_len_summary, padding='post')

### Prediction Generator

In [38]:
for i in range(len(test_text)):

  print('Review:', seq2text(test_text[i]))

  print('Original summary:', seq2summary(test_sum[i]))

  print('Predicted summary:', decode_sequence(test_text[i].reshape(1, max_len_text)))

  print('\n')

Review: have tried close regions bank account week called saturday xxxx told closed i check today xxxx voila 's magically opened i charged another service fee called today told fee posted yesterday xxxx automatically opened account back i ca n't close fee pending this absolutely ridiculous 
Original summary: bank account opened without 
Predicted summary:  great coffee but


Review: xxxx xxxx xxxx xxxx xxxx va unauthorized credit creditor i recently received copy xxxx xxxx xxxx credit report the credit report showed credit inquiry company i recall authorizing i understand n't allowed put inquiry file unless i authorized please inquiry removed credit files making difficult acquire credit send us written authorization gave access credit files ss xxxx xxxx xxxx xxxx xxxx xxxx 
Original summary: credit pull not 
Predicted summary:  disputed late payments


Review: multiple cra violations inaccurate information including names addresses phone numbers employment history pretty much entire re

In [0]:
nums = [1, 2]

letters = ['a', 'b', 'c']

cols = pd.MultiIndex.from_product([nums, letters], names=['Nums', 'Letters'])

df = pd.DataFrame(np.random.randint(100, size=(5,6)), columns=cols)

df.head()

Nums      1           2        
Letters   a   b   c   a   b   c
0        58  34  61  11   1   2
1        25  77  39  56  54  33
2        87  98   3  59  30  68
3        90  17   7  51  21  96
4        29  43  74  82  55  32

In [0]:
df.values

array([[58, 34, 61, 11,  1,  2],
       [25, 77, 39, 56, 54, 33],
       [87, 98,  3, 59, 30, 68],
       [90, 17,  7, 51, 21, 96],
       [29, 43, 74, 82, 55, 32]])

In [0]:
master_df = pd.read_csv('/content/drive/My Drive/Consumer_Complaints.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,6,11,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
proc_df = master_df.dropna(subset=['Consumer complaint narrative']).sample(100, random_state=55)

pd.set_option('max_colwidth', -1)

proc_df['Consumer complaint narrative'].head(10)

262542     This is in response to my initial complaint XXXX regarding mortgage discrimination by Chase. Despite writing to the executive board, nothing was done other than get some random person to call me and say they received my complaint. Nothing was done and they still refused my loan. No review of their practices of discrimination, no accountability, not even the bare minimum of refunding the charge they made for conducting an appraisal. \nThe bank lied to the CFPB when they said they were looking into the matter. Loan manager XXXX called me two weeks ago and said they could not honor the loan terms they provided me in writing. They are still demanding 18 months of reserves despite other big banks and credit unions confirming that 6 months was the industry norm for someone with my credit score and financial situation. \nI have a document from them with loan disclosures they have agreed to but now refuse to provide. Nothing has changed in my financial situation. Why do banks get to

In [0]:
summaries = [
             'Chase refused my loan and charged me for the appraisal.',
             'NCB Management Services is not complying with a cease and desist letter.',
             'Chase is not reporting to the credit agencies that my account is paid in full.',
             'Expenditures on a closed Wells Fargo credit card.',
             'Collection agency collecting debt that was paid.',
             ''
]

## Sample and Process Text

In [0]:
proc_df = master_df.dropna(subset=['Consumer complaint narrative']).sample(100, random_state=55)

clean = [re.sub('[^A-Za-z.,\s\']', '', nar) for nar in proc_df['Consumer complaint narrative']]

split_word_nars = [nar.split() for nar in clean]

"""Contractions Import"""

import sys
sys.path.append('/content/drive/My Drive')
from english_contractions import replace_contraction

"""End Contraction Import"""

new_words = []

for nar in split_word_nars:

  nar_words = []

  for word in nar:

    if re.search('\w+[.]', word):

      splitted = word.split('.')

      tmp_words = replace_contraction(splitted[0].lower())

      for w in tmp_words.split():

        nar_words.append(w)

      nar_words.append('.')
    
    elif re.search('\w+[,]', word):
      
      splitted = word.split(',')

      tmp_words = replace_contraction(splitted[0].lower())

      for w in tmp_words.split():

        nar_words.append(w)

      nar_words.append(',')
    
    elif re.match('[.]', word):
      
      placeholder = 1
    
    else:

      tmp_words = replace_contraction(word)

      for w in tmp_words.split():

        nar_words.append(w)

  new_words.append(nar_words)

words = [word for words in new_words for word in words]

## Tokenize Text

In [0]:
from keras.preprocessing.text import Tokenizer

num_words = 10000

tokenizer = Tokenizer(num_words=num_words, filters='')

tokenizer.fit_on_texts(words)

encoded = tokenizer.texts_to_sequences(words)

max_id = num_words

dataset_size = tokenizer.document_count

flat_encoded = [enc for encoder in encoded for enc in encoder]

print(max_id)

Using TensorFlow backend.


10000


## Split Train and Val Sets

In [0]:
train_size = dataset_size * 90 // 100

dataset = tf.data.Dataset.from_tensor_slices(flat_encoded[:train_size])

val_dataset = tf.data.Dataset.from_tensor_slices(flat_encoded[train_size:])

## Batch and Prepare Train Set

In [0]:
batch_size = 256

n_steps = 50

window_length = n_steps + 1

dataset = dataset.window(window_length, shift=1, drop_remainder=True)

dataset = dataset.flat_map(lambda window: window.batch(window_length))

dataset = dataset.shuffle(10000).batch(batch_size)

dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

dataset = dataset.map(lambda X_batch, y_batch: (tf.one_hot(X_batch, depth=max_id), y_batch))

dataset = dataset.prefetch(1)

## Batch and Prepare Val Set

In [0]:
val_dataset = val_dataset.window(window_length, shift=1, drop_remainder=True)

val_dataset = val_dataset.flat_map(lambda window: window.batch(window_length))

val_dataset = val_dataset.shuffle(10000).batch(batch_size)

val_dataset = val_dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

val_dataset = val_dataset.map(lambda X_batch, y_batch: (tf.one_hot(X_batch, depth=max_id), y_batch))

val_dataset = val_dataset.prefetch(1)

## Callbacks

In [0]:
# Early Stopping

from keras.callbacks import EarlyStopping

early_stopper = EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)

# Checkpoing Model Weights

import os

checkpoint_path = 'checkpoints/cp-{epoch:04d}.ckpt'

checkpoint_dir = os.path.dirname(checkpoint_path)

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    period=1)

## Identify Last Epoch Checkpoint

In [0]:
latest = tf.train.latest_checkpoint(checkpoint_dir)

latest

## Size of Train Set

In [0]:
steps_per_epoch = train_size // batch_size

steps_per_epoch

719

## Build and Train Model

In [0]:
import tensorflow_hub as hub

adam = tf.keras.optimizers.Adam()

model = tf.keras.models.Sequential([
                                    tf.keras.layers.GRU(256, return_sequences=True,
                                                        dropout=0.5, recurrent_dropout=0.5,
                                                        input_shape=[None, max_id]),
                                    tf.keras.layers.GRU(256, return_sequences=True,
                                                        dropout=0.5, recurrent_dropout=0.5),
                                    tf.keras.layers.Dropout(0.5),
                                    tf.keras.layers.Dense(256, activation='sigmoid'),
                                    tf.keras.layers.Dense(max_id, activation='softmax')])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.load_weights(latest)

model.fit(dataset, epochs=5, validation_data=val_dataset, callbacks=[early_stopper, checkpoint_cb])

Epoch 1/5
      1/Unknown - 2s 2s/step
Epoch 00001: saving model to checkpoints/cp-0001.ckpt


/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:842: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: 
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


      1/Unknown - 2s 2s/step

KeyboardInterrupt: ignored

In [0]:
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_30 (GRU)                 (None, None, 256)         7878144   
_________________________________________________________________
gru_31 (GRU)                 (None, None, 256)         394752    
_________________________________________________________________
dropout_20 (Dropout)         (None, None, 256)         0         
_________________________________________________________________
dense_35 (Dense)             (None, None, 256)         65792     
_________________________________________________________________
dense_36 (Dense)             (None, None, 10000)       2570000   
Total params: 10,908,688
Trainable params: 10,908,688
Non-trainable params: 0
_________________________________________________________________


## Save the Trained Model

In [0]:
drive_path = '/content/drive/'

file_path = 'My Drive/saved_keras_rnns/'

name = 'gru_50_steps_no_emb.h5'

model.save(drive_path + file_path + name)

## Save the Tokenizer

In [0]:
import pickle

tokenizer_name = 'tok_gru_50_steps_no_emb.pkl'

pickle.dump(tokenizer, open(drive_path + file_path + tokenizer_name, 'wb'))

## Load a Trained Model

In [0]:
name = 'gru_30_steps.h5'

new_model = keras.models.load_model(drive_path + file_path + name)

new_model.summary()

## Text Generation

In [0]:
def text_clean(text):

  splits = text.split(' . ')[:-1]

  new_splits = []

  for split in splits[1:]:

    word_splits = split.split(' ')
    
    word_splits[0] = word_splits[0].capitalize()
    
    word_splits[-1] = ''.join([word_splits[-1], '.'])

    joined = ' '.join(word_splits)

    new_splits.append(joined)
  
  join_split = ' '.join(new_splits)

  return join_split

def preprocessor(text):

  # X = tokenizer.texts_to_sequences(text)

  # return tf.one_hot(X, max_id) # for no embedding

  # return tokenizer.texts_to_sequences(text) # for embedding

def next_word(text, model, temperature=1):
  
  X_new = preprocessor([text])

  y_proba = model.predict(X_new)[0, -1:, :]

  rescaled_logits = tf.math.log(y_proba) / temperature

  word_id = tf.random.categorical(rescaled_logits, num_samples=1)

  return tokenizer.sequences_to_texts(word_id.numpy())[0]

def complete_text(text, model, n_words=50, temperature=0.5):

  for _ in range(n_words):

    space = [' ', next_word(text, model, temperature)]
    
    text += ''.join(space)

  return text

In [0]:
key_words = ['fcra', 'loan', 'bank', 'equifax breach', 'credit card', 'credit report']

temps = [0.15, 0.2, 0.25]

count = 0

for key_word in key_words:
  for temp in temps:
    count += 1
    print(f'{count} (kw: "{key_word}", tmp: {temp})', text_clean(complete_text(key_word, new_model_2, temperature=temp)))

1 (kw: "fcra", tmp: 0.15) I have been disputing this debt with transunion and i have never received any response from the provider of service. I have been disputing the pmi with the credit bureaus. I have been disputing this account with the credit bureaus.
2 (kw: "fcra", tmp: 0.2) I have been disputing this account with bsi financial services. I have been disputing this account with my credit report. I have been disputing this debt with the credit bureaus. I have been disputing this debt from my credit report.
3 (kw: "fcra", tmp: 0.25) I have contacted them about the process of the phone with the company and that my credit score is now. I have been trying to obtain a home and they have not been able to do so. I am not sure what happened to the.
4 (kw: "loan", tmp: 0.15) I was told that the check was rejected by seterus,. Inc. I was told that the check had bounced. I was told that the check was rejected by the bank of america.
5 (kw: "loan", tmp: 0.2) I have been a victim of identity t